In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from brian2 import *
from brian2tools import *
import numpy as np
import pandas as pd


# Customize matplotlib
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'font.size': 22,
        'ytick.color' : 'black',
        "xtick.color" : 'black',
        'axes.labelcolor' : 'black',
        'axes.edgecolor' : 'black'
	}
)

In [13]:
start_scope()

n_points = 100
n_samples = 3
vec_tauA = []
for tauA_value in np.linspace(0.1, 100, n_points):
	for n in range(n_samples):
		vec_tauA.append(tauA_value)

vec_tauA = np.array(vec_tauA)

EL = -60*mV
EE = 0*mV
EI = -80*mV
alpha = 1
tauN = 100*ms
tauI = 10*ms
tauL = 20*ms
vt = -50*mV

eqs = '''
dv/dt = ((EL - v) + gA*(EE - v) + gI*(EI - v))/tauL : volt (unless refractory)
dgA/dt = -gA/tauA : 1
dgI/dt = -gI/tauI : 1
tauA : second
'''

rate_list = [5, 10, 20]
firing_rate_ta = {'tauA':vec_tauA, 5:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples), 20:np.zeros(n_points*n_samples)}
CVISI_ta = {'tauA':vec_tauA, 5:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples), 20:np.zeros(n_points*n_samples)}

for rate in rate_list:
	print(rate, end='\t')
	for i, tauA_value in enumerate(vec_tauA):
		print(i/size(vec_tauA), end=' ')
		start_scope()
		defaultclock.dt = 0.01*ms
		G = NeuronGroup(1, eqs, threshold='v>vt', reset='v=EL', refractory=2*ms, method='rk4')
		GPE = PoissonGroup(400, rates=rate*Hz)
		GPI = PoissonGroup(100, rates=rate*Hz)

		SE = Synapses(GPE, G, 'w : 1', on_pre='gA += w')
		SI = Synapses(GPI, G, 'w : 1', on_pre='gI += w')
		SE.connect()
		SI.connect()

		G.v = EL
		SE.w = 0.1
		SI.w = 0.4

		G.tauA = tauA_value*ms

		run(5000*ms)

		M = StateMonitor(G, 'v', record=True)
		spkM = SpikeMonitor(G)
		
		run(20000*ms)

		train = spkM.spike_trains()
		ISI = np.diff(train[0])
		
		firing_rate_ta[rate][i] = spkM.num_spikes/20.0
		CVISI_ta[rate][i] = ISI.var()/ISI.mean()**2

	print()

5	0.0 0.0033333333333333335 0.006666666666666667 0.01 0.013333333333333334 0.016666666666666666 0.02 0.023333333333333334 0.02666666666666667 0.03 0.03333333333333333 0.03666666666666667 0.04 0.043333333333333335 0.04666666666666667 0.05 0.05333333333333334 0.056666666666666664 0.06 0.06333333333333334 0.06666666666666667 0.07 0.07333333333333333 0.07666666666666666 0.08 0.08333333333333333 0.08666666666666667 0.09 0.09333333333333334 0.09666666666666666 0.1 0.10333333333333333 0.10666666666666667 0.11 0.11333333333333333 0.11666666666666667 0.12 0.12333333333333334 0.12666666666666668 0.13 0.13333333333333333 0.13666666666666666 0.14 0.14333333333333334 0.14666666666666667 0.15 0.15333333333333332 0.15666666666666668 0.16 0.16333333333333333 0.16666666666666666 0.17 0.17333333333333334 0.17666666666666667 0.18 0.18333333333333332 0.18666666666666668 0.19 0.19333333333333333 0.19666666666666666 0.2 0.20333333333333334 0.20666666666666667 0.21 0.21333333333333335 0.21666666666666667 0.2

In [14]:
# Convert to pandas DataFrame
df_firing_rate = pd.DataFrame.from_dict(firing_rate_ta)
df_CVISI_ta = pd.DataFrame.from_dict(CVISI_ta)

# Save Dataframe do csv file
df_firing_rate.to_csv('firing_rate_iRate.dat')
df_CVISI_ta.to_csv('CVISI_ta_iRate.dat')

In [10]:
firing_rate_ta[5]

array([396.35, 396.75, 396.45, 395.7 , 396.35,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
      